In [1]:
import os
import numpy as np
from astropy.table import Table

In [2]:
# census from ACS 2016-2020
census = Table.read('/scratch/gpfs/chhahn/noah/floody/ACS2016_20new.csv', format='csv')

In [3]:
census = census[1:]

In [4]:
census

FIPS,Qualifying Name,File identification,State Postal Abbreviation,Summary Level,Geographic Component,Logical Record Number,US,Census Region,Census Division,State (Census Code),State (FIPS Code),County of current residence,County Subdivision (FIPS),Place (FIPS Code),Census Tract,Block Group,Consolidated City,American Indian Area/Alaska Native Area/ Hawaiian Home Land (Census),American Indian Area/Alaska Native Area/ Hawaiian Home Land (FIPS),American Indian Trust Land/Hawaiian Home Land Indicator,American Indian Tribal Subdivision (Census),American Indian Tribal Subdivision (FIPS),Alaska Native Regional Corporation (FIPS),Metropolitan and Micropolitan Statistical Area,Combined Statistical Area,Metropolitan Statistical Area- Metropolitan Division,Metropolitan Area Central City,Metropolitan/Micropolitan Indicator Flag,New England City and Town Area,New England City and Town Combined Statistical Area,New England City and Town Area Division,Urban Area,Urban Area Central Place,Current Congressional District ***,State Legislative District Upper,State Legislative District Lower,Voting District,ZIP Code Tabulation Area (3-digit),5-digit ZIP Code Tabulation Area,Subminor Civil Division (FIPS),State-School District (Elementary),State-School District (Secondary),State-School District (Unified),Urban/Rural,Principal City Indicator,Traffic Analysis Zone,Urban Growth Area,Public Use Microdata Area – 5% File,Public Use Microdata Area - 1% File,Geographic Identifier,Area Name,Tribal Tract,Tribal Block Group,Place (State FIPS + Place FIPS),Total Population,Total Population_1,Population Density (Per Sq. Mile),Area (Land),Total Population:,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,Total Population: Two or More Races,Population 25 Years and Over:,Population 25 Years and Over: Less than High School,Population 25 Years and Over: High School Graduate or More (Includes Equivalency),Population 25 Years and Over: Some College or More,Population 25 Years and Over: Bachelor's Degree or More,Population 25 Years and Over: Master's Degree or More,Population 25 Years and Over: Professional School Degree or More,Population 25 Years and Over: Doctorate Degree,Renter-Occupied Housing Units,Renter-Occupied Housing Units: Less than High School Graduate,Renter-Occupied Housing Units: High School Graduate (Including Equivalency),Renter-Occupied Housing Units: Some College or Associate's Degree,Renter-Occupied Housing Units: Bachelor's Degree or Higher,Median Household Income (In 2020 Inflation Adjusted Dollars),Median Household Income (In 2020 Inflation Adjusted Dollars):,Median Household Income (In 2020 Inflation Adjusted Dollars): White Alone Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): Black or African American Alone Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): American Indian and Alaska Native Alone Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): Asian Alone,Median Household Income (In 2020 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): Some Other Race Alone Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): Two or More Races Householder,Median Household Income (In 2020 Inflation Adjusted Dollars): Hispanic or Latino Householder,"Median Household Income (In 2020 Inflation Adjusted Dollars): White Alone Householder, Not Hispanic or Latino",Households:,Households: with Earnings,Households: No Earnings,Per Capita Income (In 2020 Inflation Adjusted Dollars),Lowest Quintile,Second Quintile,Third Quintile,Fourth Quintile,Lower Limit of Top 5 Percent,Occupied Housing Units:,Occupied Housing Units: Owner Occupied,Occupied Housing Un

compile 
1. median_household_income
2. population
3. renter_fraction 
4. educated_fraction
5. white_fraction

In [7]:
income = np.array(census['Median Household Income (In 2020 Inflation Adjusted Dollars)']).astype(float)
income = Table.MaskedColumn(income, dtype=float)

population = np.array(census['Total Population']).astype(float)
population = Table.MaskedColumn(population, dtype=float)

In [9]:
renter_fraction = (np.array(census['Total Population in Occupied Housing Units: Renter Occupied'].data).astype(float) / 
                   np.array(census['Total Population'].data).astype(float))
renter_fraction = Table.MaskedColumn(renter_fraction, dtype=float)
renter_fraction.mask = np.array(census['Total Population'].data).astype(float) == 0.

/tmp/ipykernel_2701335/1273455776.py:1: RuntimeWarning: invalid value encountered in divide
  renter_fraction = (np.array(census['Total Population in Occupied Housing Units: Renter Occupied'].data).astype(float) /


In [10]:
educated_fraction = (
    np.array(census['Population 25 Years and Over: Bachelor\'s Degree or More'].data).astype(float) / 
    np.array(census['Population 25 Years and Over:'].data).astype(float))

educated_fraction = Table.MaskedColumn(educated_fraction, dtype=float)
educated_fraction.mask = np.array(census['Population 25 Years and Over:'].data).astype(float) == 0.

/tmp/ipykernel_2701335/1386555845.py:2: RuntimeWarning: invalid value encountered in divide
  np.array(census['Population 25 Years and Over: Bachelor\'s Degree or More'].data).astype(float) /


In [11]:
white_fraction = (
    np.array(census['Total Population: White Alone'].data).astype(float) / 
    np.array(census['Total Population'].data).astype(float))

white_fraction = Table.MaskedColumn(white_fraction, dtype=float)
white_fraction.mask = np.array(census['Total Population'].data).astype(float) == 0.

/tmp/ipykernel_2701335/2063348824.py:2: RuntimeWarning: invalid value encountered in divide
  np.array(census['Total Population: White Alone'].data).astype(float) /


In [12]:
combined_mask = income.mask | population.mask | renter_fraction.mask | educated_fraction.mask | white_fraction.mask
print('%.2f masked' % np.mean(~combined_mask)) 

0.99 masked


In [13]:
output = Table()
output['ZIPCODE'] = np.array([int(name.split(' ')[1]) for name in census['Qualifying Name']])[~combined_mask]
output['income'] = income[~combined_mask]
output['population'] = population[~combined_mask]
output['renter_fraction'] = renter_fraction[~combined_mask]
output['educated_fraction'] = educated_fraction[~combined_mask]
output['white_fraction'] = white_fraction[~combined_mask]

In [14]:
output

ZIPCODE,income,population,renter_fraction,educated_fraction,white_fraction
int64,float64,float64,float64,float64,float64
601,14398.0,16773.0,0.36958206641626423,0.15715968366145044,0.7294461336672032
602,16771.0,37083.0,0.20858614459455815,0.20502353633421594,0.7652563169107138
603,15786.0,45652.0,0.4094891790063962,0.23260691164402297,0.7800753526680102
606,14980.0,6231.0,0.26624939817043813,0.10549354344495514,0.5069812229176697
610,20167.0,26502.0,0.22054939249867936,0.17037573750129387,0.7555278846879481
612,19402.0,58771.0,0.3562471286859165,0.2527193274117703,0.6665362168416396
616,18903.0,9688.0,0.19405450041288191,0.21007371007371006,0.6864161849710982
617,17702.0,23771.0,0.2786168019856127,0.22267110266159695,0.7015270708005553
622,15640.0,6505.0,0.19815526518063029,0.20403787743433982,0.26379707916986933


In [15]:
output.write('/scratch/gpfs/chhahn/noah/floody/floody.acs_census.hdf5', format='hdf5', overwrite=True)

/home/chhahn/.conda/envs/sbi/lib/python3.10/site-packages/astropy/io/misc/hdf5.py:281: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(
